# The Nature Conservancy Fisheries Monitoring
[The Nature Conservancy Fisheries Monitoring](https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring) 

## Action Plan
### 1. Data Preparation and Preprocessing
### 2. Finetune and Train Model
### 2b. Data Augmentation parameter tuning
### 3. Generate and Validate Predictions 
### 4. Submit predictions to Kaggle

In [1]:
%pwd

'D:\\Dropbox\\Ted 2017\\11A_技術\\[TREND] Data Science and ML\\[Tutorials] Kaggle\\[Competitions] Kaggle Competitions\\2016-10_ImgClass_The Nature Fisheries Monitoring\\code'

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\code
D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\code/../input/


In [3]:
#Allow relative imports to directories above this folder/
sys.path.insert(1,os.path.join(sys.path[0], '..'))

#import modules
import utils
from utils import *
# from vgg16_ted import vgg16BN
%matplotlib inline

Using TensorFlow backend.


#### Setup Paths

In [4]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' # + '/sample/' 
result_path = path + 'results/'
train_path = path + 'train/'
valid_path = path + 'valid/'
test_path = path + 'test/'
model_path = path + 'models/'

D:\Dropbox\Ted 2017\11A_技術\[TREND] Data Science and ML\[Tutorials] Kaggle\[Competitions] Kaggle Competitions\2016-10_ImgClass_The Nature Fisheries Monitoring\input


In [5]:
batch_size = 32 # batch_size = 64

In [6]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

(trn_classes, val_classes, trn_labels, val_labels, 
    filenames, val_filenames, test_filenames) = get_classes(path)

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [ ]:
trn_data = get_data(train_path)
val_data = get_data(valid_path)
test_data = get_data(test_path)

In [ ]:
save_array(result_path+'trn_data', trn_data)
save_array(result_path+'val_data', val_data)
save_array(result_path+'test_data', test_data)

In [ ]:
trn_data = load_array(result_path+'trn_data')
val_data = load_array(result_path+'val_data')
test_data = load_array(result_path+'test_data')

Sometimes it's helpful to have just the filenames, without the path.

In [7]:
"""
# [Linux Environment]
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
"""
# [Windows Environment]
raw_filenames = [f.split('\\')[-1] for f in filenames]
raw_val_filenames = [f.split('\\')[-1] for f in val_filenames]
raw_test_filenames = [f.split('\\')[-1] for f in test_filenames]

In [8]:
len(raw_filenames), len(raw_val_filenames), len(raw_test_filenames)

(3277, 500, 1000)

In [9]:
input_shape = next(batches)[0].shape[1:]
print(input_shape)

(3, 224, 224)


## 2. Finetune and Train Model

### Linear model
#### always start with the simpliest model (ex: linear model here) to know the baseline

In [11]:
def get_lm():
    model = Sequential([
        BatchNormalization(input_shape=input_shape),
        Flatten(),
        Dense(8, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
lm = get_lm()

In [13]:
lm.fit_generator(batches, epochs=2,
                 steps_per_epoch=int(math.ceil(batches.n/batches.batch_size)), 
                 validation_data=val_batches,
                 validation_steps=int(math.ceil(val_batches.n/val_batches.batch_size)))

Epoch 1/2
103/103 [==============================] - 168s - loss: 8.5085 - acc: 0.4561 - val_loss: 8.1374 - val_acc: 0.4920
Epoch 2/2
103/103 [==============================] - 126s - loss: 8.0374 - acc: 0.4985 - val_loss: 8.3212 - val_acc: 0.4820


In [14]:
lm.lr = 0.1

In [15]:
lm.fit_generator(batches, epochs=8,
                 steps_per_epoch=int(math.ceil(batches.n/batches.batch_size)), 
                 validation_data=val_batches,
                 validation_steps=int(math.ceil(val_batches.n/val_batches.batch_size)))

Epoch 1/8
103/103 [==============================] - 162s - loss: 7.9515 - acc: 0.5034 - val_loss: 7.6540 - val_acc: 0.5220
Epoch 2/8
103/103 [==============================] - 181s - loss: 7.3841 - acc: 0.5393 - val_loss: 7.3726 - val_acc: 0.5400
Epoch 3/8
103/103 [==============================] - 282s - loss: 7.6902 - acc: 0.5210 - val_loss: 7.3884 - val_acc: 0.5380
Epoch 4/8
103/103 [==============================] - 252s - loss: 7.4772 - acc: 0.5343 - val_loss: 7.1564 - val_acc: 0.5560
Epoch 5/8
103/103 [==============================] - 266s - loss: 7.2965 - acc: 0.5454 - val_loss: 7.2572 - val_acc: 0.5480
Epoch 6/8
103/103 [==============================] - 133s - loss: 7.5621 - acc: 0.5302 - val_loss: 6.9682 - val_acc: 0.5660
Epoch 7/8
103/103 [==============================] - 89s - loss: 7.5811 - acc: 0.5283 - val_loss: 6.9507 - val_acc: 0.5680
Epoch 8/8
103/103 [==============================] - 153s - loss: 7.2525 - acc: 0.5488 - val_loss: 7.1389 - val_acc: 0.5560


### Basic VGG

First we create a simple fine-tuned VGG model to be our starting point.

In [ ]:
from vgg16bn_ted import Vgg16BN
model = vgg_ft_bn(8)